<a href="https://colab.research.google.com/github/costpetrides/FAIRMODE-WG5/blob/main/Python/XGBSF_O3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 77.3 MB/s eta 0:00:00


In [2]:
pip install netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 66.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import netCDF4 as nc
import xgboost as xgb
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import BallTree
from tqdm import tqdm

# === Load CSV Data (Station Measurements) ===
csv_file_path = "baseO3nearest_grid.csv"
df = pd.read_csv(csv_file_path)

# Compute Bias (Observed - Modeled)
df["bias"] = df["SURF_ppb_O3"] - df["nearest_SURF_ppb_O3"]

# === Load NetCDF Data (Grid Model) ===
netcdf_path = "BaseCase_PERT_O3_YEARLY.nc"
dataset = nc.Dataset(netcdf_path, "r")

lon = dataset.variables["lon"][:]
lat = dataset.variables["lat"][:]
o3_modeled = dataset.variables["SURF_ppb_O3"][0, :, :]

# Convert Degrees to Radians for Haversine Distance
lon_rad = np.radians(lon)
lat_rad = np.radians(lat)

# === Create Meshgrid for Grid Points ===
lon_mesh, lat_mesh = np.meshgrid(lon_rad, lat_rad)
grid_points = np.column_stack([lat_mesh.ravel(), lon_mesh.ravel()])

# === Prepare Station Data (Convert to Radians) ===
station_points = np.column_stack([
    np.radians(df["nearest_grid_lat"].values),
    np.radians(df["nearest_grid_lon"].values)
])
station_o3 = df["nearest_SURF_ppb_O3"].values  # Modeled O₃ at stations
station_bias = df["bias"].values  # Observed bias

# === BallTree for Nearest Neighbor Search ===
tree = BallTree(station_points, metric="haversine")

# === Function: Compute Spatial Features ===
def compute_spatial_features(points, station_points, station_o3, station_bias, k=5):
    """Finds nearest k stations and computes mean, min, max, variance, and IDW-weighted O₃."""
    dists, idxs = tree.query(points, k=k)

    mean_o3 = np.mean(station_o3[idxs], axis=1)
    min_o3 = np.min(station_o3[idxs], axis=1)
    max_o3 = np.max(station_o3[idxs], axis=1)
    var_o3 = np.var(station_o3[idxs], axis=1)
    mean_bias = np.mean(station_bias[idxs], axis=1)

    weights = 1 / (dists + 1e-6)
    weights /= np.sum(weights, axis=1, keepdims=True)

    idw_o3 = np.sum(weights * station_o3[idxs], axis=1)

    return mean_o3, min_o3, max_o3, var_o3, idw_o3, mean_bias

# === Compute Spatial Features for Stations ===
mean_o3_s, min_o3_s, max_o3_s, var_o3_s, idw_o3_s, mean_bias_s = compute_spatial_features(
    station_points, station_points, station_o3, station_bias, k=5
)

# Construct Feature Matrix for XGBoost Training
X_train = np.column_stack([
    df["nearest_grid_lon"].values,
    df["nearest_grid_lat"].values,
    df["nearest_SURF_ppb_O3"].values,
    mean_o3_s, min_o3_s, max_o3_s, var_o3_s, idw_o3_s, mean_bias_s
])
y_train = station_bias

# === Hyperparameter Grid for XGBoost ===
param_grid = {
    "n_estimators": [100, 300],
    "max_depth": [10, 15],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 0.1, 0.2],
}

param_list = list(ParameterGrid(param_grid))

# === Grid Search with RMSE, MAE, R² Calculation ===
best_rmse = float("inf")
best_params = None
results = []

print("\nPerforming Hyperparameter Optimization (XGBoost)...")

for params in tqdm(param_list, desc="Grid Search Progress", unit="combination"):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_rmses, fold_maes, fold_r2s = [], [], []

    for train_idx, test_idx in kf.split(X_train):
        X_tr, X_te = X_train[train_idx], X_train[test_idx]
        y_tr, y_te = y_train[train_idx], y_train[test_idx]

        model = xgb.XGBRegressor(**params, random_state=42, n_jobs=-1)
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)

        fold_rmse = np.sqrt(mean_squared_error(y_te, y_pred))
        fold_mae = mean_absolute_error(y_te, y_pred)
        fold_r2 = r2_score(y_te, y_pred)

        fold_rmses.append(fold_rmse)
        fold_maes.append(fold_mae)
        fold_r2s.append(fold_r2)

    mean_rmse = np.mean(fold_rmses)
    mean_mae = np.mean(fold_maes)
    mean_r2 = np.mean(fold_r2s)

    results.append((params, mean_rmse, mean_mae, mean_r2))

    if mean_rmse < best_rmse:
        best_rmse = mean_rmse
        best_params = params

print(f"\n Best Parameters Found: {best_params}")
print(f"Best Model Score (RMSE): {best_rmse:.4f}")

# Train Final XGBoost Model with Best Parameters
best_model = xgb.XGBRegressor(**best_params, random_state=42, n_jobs=-1)
best_model.fit(X_train, y_train)

# === Compute LOSO RMSE ===
loso_predictions, loso_actuals = [], []

for i in tqdm(range(len(X_train)), desc="LOSO Progress"):
    X_tr = np.delete(X_train, i, axis=0)
    y_tr = np.delete(y_train, i)

    rf_loso = xgb.XGBRegressor(**best_params, random_state=42, n_jobs=-1)
    rf_loso.fit(X_tr, y_tr)

    X_test = X_train[i].reshape(1, -1)
    y_pred = rf_loso.predict(X_test)[0]

    loso_predictions.append(y_pred)
    loso_actuals.append(y_train[i])

loso_rmse = np.sqrt(mean_squared_error(loso_actuals, loso_predictions))
loso_mae = mean_absolute_error(loso_actuals, loso_predictions)
loso_r2 = r2_score(loso_actuals, loso_predictions)

print(f"\nLOSO RMSE: {loso_rmse:.4f}, MAE: {loso_mae:.4f}, R²: {loso_r2:.4f}")

# === Predict Bias on the Grid ===
print("\n Predicting Bias on the Grid with Optimized XGBoost...")
interpolated_bias_xgb = best_model.predict(X_train).reshape(o3_modeled.shape)

# === Save Bias Correction to NetCDF ===
xgb_bias_netcdf_path = "BaseCase_O3_Y_XGB.nc"

with nc.Dataset(xgb_bias_netcdf_path, "w", format="NETCDF4") as bias_nc:
    bias_nc.createDimension("lat", lat.shape[0])
    bias_nc.createDimension("lon", lon.shape[0])

    lat_var = bias_nc.createVariable("lat", "f4", ("lat",))
    lon_var = bias_nc.createVariable("lon", "f4", ("lon",))
    bias_var = bias_nc.createVariable("Interpolated_Bias_XGB", "f4", ("lat", "lon"))

    lat_var[:] = lat
    lon_var[:] = lon
    bias_var[:, :] = interpolated_bias_xgb

print(f"\n XGBoost Bias Interpolation Saved: {xgb_bias_netcdf_path}")